# North Korea provocations database

In [ ]:
# !/opt/anaconda3/bin/python -m pip install nb-black

In [2]:
%load_ext lab_black

In [3]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt

In [4]:
%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat
import dateutil

In [5]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
from datetime import datetime, date

In [6]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.set_option("display.max_colwidth", None)

### Download latest version of the database from here

In [7]:
# https://beyondparallel.csis.org/database-north-korean-provocations/

### Import data

In [8]:
src = pd.read_csv("input/Provocations New.csv")

In [9]:
src.columns = (
    src.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace(")", "")
    .str.replace("-", "_")
)

In [10]:
src.head()

,date,type,event,description,resources
0,1958-02-16,Other Provocation,Hijacking of South Korean Plane,North Korean agents hijacked a South Korean plane en route from Busan to Seoul. Twenty-eight kidnapped passengers taken to Pyongyang were eventually released but eight other passengers were never returned.,"https://goo.gl/Xn9Wr4 The US Military Experience in Korea 1871-1982, page 108."
1,1959-06-15,Other Provocation,NK Fighter Jets Attack U.S. Navy Plane,Two North Korean jet fighters (MiG) attacked and damaged a U.S. navy patrol plane (P4M Mercator) off the eastern coast of North Korea. The pilot returned safely to base in Japan's Honshu Island.,"https://goo.gl/UFh93r The US Military Experience in Korea 1871-1982, page 109."
2,1960-07-30,Other Provocation,Exchange of Fire and Sinking of NK Vessel,A South Korean destroyer sinks a North Korean patrol vessel off the eastern coast killing at least 14 crewmen. This was the first naval clash between the two Koreas since the signing of the Armistice in 1953.,"https://goo.gl/NEgR7x The US Military Experience in Korea 1871-1982, page 109."
3,1962-11-20,Other Provocation,Exchange of Fire at DMZ,NK troops attacked a UN observation post 7 miles southeast of Panmunjom and an armed clash ensues.,"http://www.cfr.org/content/publications/attachments/Korean_Peninsula_Clashes.pdf; The US Military Experience in Korea 1871-1982, page 113."
4,1963-07-29,Other Provocation,Exchange of Fire at DMZ,A group of at least seven North Korean soldiers crossed the DMZ and killed two U.S. soldiers and wounded one during an attack on a jeep with a machine gun and hand grenade. U.S. and South Korean patrols tracked and killed two of the infiltrators.,"https://goo.gl/8PXjzE; The US Military Experience in Korea 1871-1982, page 113."


### Parse dates

In [11]:
src["date"] = pd.to_datetime(src["date"])
src["year"] = src["date"].dt.year
src["month"] = src["date"].dt.month
src["day"] = src["date"].dt.day
src["day"] = src["date"].dt.day_name()

In [16]:
src.groupby(["type", "year"]).agg({"date": "count"}).reset_index().head()

,type,year,date
0,Missile Provocation,1984,2
1,Missile Provocation,1985,1
2,Missile Provocation,1990,2
3,Missile Provocation,1991,2
4,Missile Provocation,1992,1


In [13]:
missiles = src[src["type"] == "Missile Provocation"]

In [14]:
missiles.sort_values("date", ascending=False).head()

,date,type,event,description,resources,year,month,day
294,2021-03-21,Missile Provocation,Anti-Ship Cruise Missile Launch,"On March 21, 2021 at around 6:36am (KST), North Korea fired two cruise missiles from the western port county of Onchon.",https://en.yna.co.kr/view/AEN20210324000555325?section=news,2021,3,Sunday
293,2020-07-06,Missile Provocation,Anti-Ship Cruise Missile Launch,"On July 7, North Korea launched ship-to-ship missile from a ship in te sea between Korea and Japan that flew about 100 km. South Korean authorities saw the test as part of North Korea's regular maritime exercise.",https://en.yna.co.kr/view/AEN20200730003600325,2020,7,Monday
291,2020-04-14,Missile Provocation,Anti-Ship Cruise Missile Launch,"On April 14, 2020 at 7:00am (KST), North Korea launched cruise missiles near the town of Munchon and air-to-ground missiles from fighter jets near Wonsan for more than 40 minutes. The projectiles travelled around 150 km towards the sea between Korea and Japan.",https://en.yna.co.kr/view/AEN20200414006452325?section=nk/nk,2020,4,Tuesday
290,2020-03-21,Missile Provocation,Short-range Missile Launch,"On March 21, 2020 at 6:45am and 6:50pm (KST), North Korea launched two projectiles from the county of Sonchon in North Pyongan Province towards the sea between Korea and Japan. The projectiles traveled a distance of 410km (255 miles) at a maximum altitude of 50km (31 miles) and closely resemble the KN-24 short-range ballistic missile system.",https://en.yna.co.kr/view/AEN20200321000454325?section=nk/nk,2020,3,Saturday
289,2020-03-06,Missile Provocation,Short-range Missile Launch,"On March 9. 2020 at 7:36am (KST), North Korea launched three short-range projectiles from the Sondok, South Hamgyong Province area towards the sea between Korea and Japan. The projectiles traveled a distance of 200 km at a maximum altitude of 50 km, and closely resemble the KN-25 multiple rocket launch system.",https://en.yna.co.kr/view/AEN20200309000455325,2020,3,Friday


In [15]:
missiles.groupby(["event"]).agg({"date": "count"}).reset_index()

,event,date
0,Anti-Ship Cruise Missile Launch,18
1,Ballistic Missile Launch (type unknown),2
2,Inter-Continental Ballistic Missile/Short-Range Missile Launches,1
3,Inter-continental Ballistic Missile Launch,2
4,Intermediate-range Ballistic Missile Launch,12
5,Joint Missile Test with China,1
6,Long-range Surface-to-Air Missile Launch,1
7,Medium-range Ballistic Missile Launch,13
8,Satellite Launch (Other Provocation),4
9,Short-range Ballistic Missile Launch,38
